# String Matching Algorithms - Logic & Dry Runs

## **1. Naive String Matching (Brute Force)**

### **Logic & Intuition**

**Concept:** Slide the pattern over the text one position at a time and check for a match at each position.

**Think of it like:** Looking for a word in a book by checking every possible position character by character.

### **Code**
```python
def naive_string_match(text, pattern):
    n = len(text)
    m = len(pattern)
    matches = []
    
    # Try each position in text
    for i in range(n - m + 1):
        match = True
        
        # Check if pattern matches at position i
        for j in range(m):
            if text[i + j] != pattern[j]:
                match = False
                break
        
        if match:
            matches.append(i)
    
    return matches
```

### **Dry Run: `text = "AABAACAADAABAABA"`, `pattern = "AABA"`**

```
Text:    A A B A A C A A D A A B A A B A
Index:   0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15

Pattern: A A B A (length = 4)

We check positions 0 to (16-4) = 12
```

**Position i=0:**
```
Text:    [A A B A] A C A A D A A B A A B A
Pattern:  A A B A

Compare:
  text[0] = 'A' vs pattern[0] = 'A' ✓
  text[1] = 'A' vs pattern[1] = 'A' ✓
  text[2] = 'B' vs pattern[2] = 'B' ✓
  text[3] = 'A' vs pattern[3] = 'A' ✓

Match found at position 0!
matches = [0]
```

**Position i=1:**
```
Text:    A [A B A A] C A A D A A B A A B A
Pattern:    A A B A

Compare:
  text[1] = 'A' vs pattern[0] = 'A' ✓
  text[2] = 'B' vs pattern[1] = 'A' ✗

No match, move to next position
```

**Position i=2:**
```
Text:    A A [B A A C] A A D A A B A A B A
Pattern:      A A B A

Compare:
  text[2] = 'B' vs pattern[0] = 'A' ✗

No match, move to next position
```

**Position i=3:**
```
Text:    A A B [A A C A] A D A A B A A B A
Pattern:        A A B A

Compare:
  text[3] = 'A' vs pattern[0] = 'A' ✓
  text[4] = 'A' vs pattern[1] = 'A' ✓
  text[5] = 'C' vs pattern[2] = 'B' ✗

No match
```

**Continuing this process...**

**Position i=9:**
```
Text:    A A B A A C A A D [A A B A] A B A
Pattern:                      A A B A

Compare:
  text[9] = 'A' vs pattern[0] = 'A' ✓
  text[10] = 'A' vs pattern[1] = 'A' ✓
  text[11] = 'B' vs pattern[2] = 'B' ✓
  text[12] = 'A' vs pattern[3] = 'A' ✓

Match found at position 9!
matches = [0, 9]
```

**Position i=12:**
```
Text:    A A B A A C A A D A A B [A A B A]
Pattern:                            A A B A

Compare:
  text[12] = 'A' vs pattern[0] = 'A' ✓
  text[13] = 'A' vs pattern[1] = 'A' ✓
  text[14] = 'B' vs pattern[2] = 'B' ✓
  text[15] = 'A' vs pattern[3] = 'A' ✓

Match found at position 12!
matches = [0, 9, 12]
```

**Final Result:** Pattern found at positions `[0, 9, 12]`

**Time Complexity:** O(n×m) where n = text length, m = pattern length

---

## **2. KMP Algorithm (Knuth-Morris-Pratt)**

### **Logic & Intuition**

**Key Idea:** Don't re-check characters we already know match. Use information from previous comparisons to skip unnecessary checks.

**The Secret Weapon: LPS Array (Longest Proper Prefix which is also Suffix)**

**What is LPS?**
- For pattern "AABAAB", at each position, find the longest prefix that is also a suffix
- This tells us where to jump back to when a mismatch occurs

**Think of it like:** If you're reading and see "AABAA" then mismatch, you don't start over - you know "AA" already matches at the beginning.

### **Code**

```python
def kmp_search(text, pattern):
    n = len(text)
    m = len(pattern)
    
    # Step 1: Build LPS array
    lps = compute_lps(pattern)
    
    # Step 2: Search using LPS
    matches = []
    i = 0  # index for text
    j = 0  # index for pattern
    
    while i < n:
        if pattern[j] == text[i]:
            i += 1
            j += 1
        
        if j == m:
            matches.append(i - j)
            j = lps[j - 1]
        elif i < n and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]  # Don't match lps[0..lps[j-1]], skip ahead
            else:
                i += 1
    
    return matches

def compute_lps(pattern):
    m = len(pattern)
    lps = [0] * m
    length = 0  # length of previous longest prefix suffix
    i = 1
    
    while i < m:
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1
    
    return lps
```

### **Dry Run Part 1: Building LPS Array for pattern = "AABAAB"**

```
Pattern: A A B A A B
Index:   0 1 2 3 4 5
```

**Initial State:**
```
lps = [0, 0, 0, 0, 0, 0]
length = 0 (length of longest proper prefix suffix)
i = 1 (start from second character)
```

**Step-by-step LPS Construction:**

**i=1:**
```
Compare pattern[1]='A' with pattern[0]='A'
Match! 
  length = 1
  lps[1] = 1
  i = 2

lps = [0, 1, 0, 0, 0, 0]
      A A
      ↑ ↑
     prefix of length 1 matches
```

**i=2:**
```
Compare pattern[2]='B' with pattern[1]='A'
No match!
  length was 1, so check lps[1-1] = lps[0] = 0
  length = 0
  lps[2] = 0
  i = 3

lps = [0, 1, 0, 0, 0, 0]
```

**i=3:**
```
Compare pattern[3]='A' with pattern[0]='A'
Match!
  length = 1
  lps[3] = 1
  i = 4

lps = [0, 1, 0, 1, 0, 0]
      A A B A
      ↑     ↑
     prefix of length 1 matches
```

**i=4:**
```
Compare pattern[4]='A' with pattern[1]='A'
Match!
  length = 2
  lps[4] = 2
  i = 5

lps = [0, 1, 0, 1, 2, 0]
      A A B A A
      ↑ ↑   ↑ ↑
     prefix of length 2 matches
```

**i=5:**
```
Compare pattern[5]='B' with pattern[2]='B'
Match!
  length = 3
  lps[5] = 3
  i = 6

lps = [0, 1, 0, 1, 2, 3]
      A A B A A B
      ↑ ↑ ↑ ↑ ↑ ↑
     prefix of length 3 matches
```

**Final LPS Array:** `[0, 1, 0, 1, 2, 3]`

**What LPS means:**
```
Pattern:  A A B A A B
LPS:      0 1 0 1 2 3

Position 0: No proper prefix/suffix → 0
Position 1: "A" is both prefix and suffix → 1
Position 2: No match → 0
Position 3: "A" is both prefix and suffix → 1
Position 4: "AA" is both prefix and suffix → 2
Position 5: "AAB" is both prefix and suffix → 3
```

---

### **Dry Run Part 2: Searching with KMP**

**Text = "AABAACAADAABAABA"**, **Pattern = "AABA"**

**First, compute LPS for "AABA":**
```
Pattern: A A B A
LPS:     0 1 0 1
```

**Now search:**

```
Text:    A A B A A C A A D A A B A A B A
Index:   0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15

i = text pointer, j = pattern pointer
```

**Initial:** `i=0, j=0`

**Step 1-4: Match at position 0**
```
i=0, j=0: text[0]='A' == pattern[0]='A' ✓ → i=1, j=1
i=1, j=1: text[1]='A' == pattern[1]='A' ✓ → i=2, j=2
i=2, j=2: text[2]='B' == pattern[2]='B' ✓ → i=3, j=3
i=3, j=3: text[3]='A' == pattern[3]='A' ✓ → i=4, j=4

j == 4 (pattern length) → Match found at position 0!
Reset: j = lps[3] = 1
```

**Step 5: Continue from where we are**
```
i=4, j=1: text[4]='A' == pattern[1]='A' ✓ → i=5, j=2
```

**Step 6: Mismatch**
```
i=5, j=2: text[5]='C' != pattern[2]='B' ✗
j != 0, so j = lps[1] = 1
```

**Step 7: Still mismatch**
```
i=5, j=1: text[5]='C' != pattern[1]='A' ✗
j != 0, so j = lps[0] = 0
```

**Step 8: Still mismatch**
```
i=5, j=0: text[5]='C' != pattern[0]='A' ✗
j == 0, so i = 6
```

**Visual representation of the skip:**
```
Text:    A A B A [A C] A A D...
Pattern: A A B A
                 A A B A (naive would restart here)
             A A B A (KMP knows "A" matches, jumps here)
```

**Continuing...**

```
i=6, j=0: text[6]='A' == pattern[0]='A' ✓ → i=7, j=1
i=7, j=1: text[7]='A' == pattern[1]='A' ✓ → i=8, j=2
i=8, j=2: text[8]='D' != pattern[2]='B' ✗
  j = lps[1] = 1

i=8, j=1: text[8]='D' != pattern[1]='A' ✗
  j = lps[0] = 0

i=8, j=0: text[8]='D' != pattern[0]='A' ✗
  i = 9

i=9, j=0: text[9]='A' == pattern[0]='A' ✓ → i=10, j=1
i=10, j=1: text[10]='A' == pattern[1]='A' ✓ → i=11, j=2
i=11, j=2: text[11]='B' == pattern[2]='B' ✓ → i=12, j=3
i=12, j=3: text[12]='A' == pattern[3]='A' ✓ → i=13, j=4

j == 4 → Match found at position 9!
Reset: j = lps[3] = 1

i=13, j=1: text[13]='A' == pattern[1]='A' ✓ → i=14, j=2
i=14, j=2: text[14]='B' == pattern[2]='B' ✓ → i=15, j=3
i=15, j=3: text[15]='A' == pattern[3]='A' ✓ → i=16, j=4

j == 4 → Match found at position 12!
```

**Final Result:** Matches at `[0, 9, 12]`

**Time Complexity:** O(n + m) - Much better than naive O(n×m)!

**Key Advantage:** When we have a mismatch, we don't go all the way back - we use LPS to skip ahead intelligently.

---

## **3. Rabin-Karp Algorithm (Rolling Hash)**

### **Logic & Intuition**

**Key Idea:** Use hashing to quickly compare pattern with text substrings. Instead of comparing character by character, compare hash values.

**The Magic: Rolling Hash**
- Calculate hash of first window
- "Roll" the hash by removing first character and adding next character
- Only when hashes match, verify character by character (to avoid false positives)

**Think of it like:** Instead of reading every word in a book, you check the "fingerprint" first.

### **Code**

```python
def rabin_karp(text, pattern):
    n = len(text)
    m = len(pattern)
    d = 256  # number of characters in alphabet
    q = 101  # a prime number for modulo
    
    matches = []
    h = 1  # hash value for removing leading digit
    pattern_hash = 0
    text_hash = 0
    
    # Calculate h = d^(m-1) % q
    for i in range(m - 1):
        h = (h * d) % q
    
    # Calculate hash for pattern and first window of text
    for i in range(m):
        pattern_hash = (d * pattern_hash + ord(pattern[i])) % q
        text_hash = (d * text_hash + ord(text[i])) % q
    
    # Slide the pattern over text
    for i in range(n - m + 1):
        # Check if hash values match
        if pattern_hash == text_hash:
            # Verify character by character (to handle hash collisions)
            if text[i:i+m] == pattern:
                matches.append(i)
        
        # Calculate hash for next window (rolling hash)
        if i < n - m:
            text_hash = (d * (text_hash - ord(text[i]) * h) + ord(text[i + m])) % q
            
            # Handle negative hash
            if text_hash < 0:
                text_hash += q
    
    return matches
```

### **Dry Run: `text = "ABCCDDAEFG"`, `pattern = "CDD"`**

**Parameters:**
```
d = 256 (alphabet size)
q = 101 (prime for modulo)
m = 3 (pattern length)
```

**Step 1: Calculate h = d^(m-1) % q**
```
h = 256^2 % 101
h = 65536 % 101
h = 64
```

**Step 2: Calculate hash for pattern "CDD"**
```
pattern_hash = 0

i=0: pattern_hash = (256 * 0 + ord('C')) % 101
              = (0 + 67) % 101 = 67

i=1: pattern_hash = (256 * 67 + ord('D')) % 101
              = (17152 + 68) % 101
              = 17220 % 101 = 64

i=2: pattern_hash = (256 * 64 + ord('D')) % 101
              = (16384 + 68) % 101
              = 16452 % 101 = 94

pattern_hash = 94
```

**Step 3: Calculate hash for first window "ABC"**
```
text_hash = 0

i=0: text_hash = (256 * 0 + ord('A')) % 101 = 65
i=1: text_hash = (256 * 65 + ord('B')) % 101 = (16640 + 66) % 101 = 71
i=2: text_hash = (256 * 71 + ord('C')) % 101 = (18176 + 67) % 101 = 38

text_hash = 38
```

**Step 4: Slide and compare**

**Position i=0: Window "ABC"**
```
text_hash = 38
pattern_hash = 94
38 != 94 → No match

Calculate hash for next window:
  Remove 'A': text_hash = (38 - 65*64) % 101
  Add 'C':    text_hash = (256 * result + 67) % 101
  
  text_hash = (256 * (38 - 4160) + 67) % 101
  text_hash = (256 * (-4122) + 67) % 101
  text_hash needs to handle negative...
  text_hash = (256 * (38 - 4160 % 101) + 67) % 101
  
Let me recalculate properly:
  text_hash = 38 - (ord('A') * h) % q
           = 38 - (65 * 64) % 101
           = 38 - 4160 % 101
           = 38 - 19
           = 19
  
  text_hash = (d * 19 + ord('C')) % q
           = (256 * 19 + 67) % 101
           = (4864 + 67) % 101
           = 4931 % 101
           = 75
```

**Position i=1: Window "BCC"**
```
text_hash = 75
pattern_hash = 94
75 != 94 → No match

Roll to next window "CCD":
  Remove 'B': text_hash = (75 - 66*64) % 101 = (75 - 4224 % 101)
  text_hash = (75 - 81) % 101
  Handle negative: text_hash = (75 - 81 + 101) % 101 = 95
  
  Add 'D': text_hash = (256 * 95 + 68) % 101
        = (24320 + 68) % 101
        = 24388 % 101
        = 64
```

**Position i=2: Window "CCD"**
```
text_hash = 64
pattern_hash = 94
64 != 94 → No match

Roll to next window "CDD":
  Remove 'C': text_hash = (64 - 67*64) % 101
  text_hash = (64 - 4288 % 101) % 101
  text_hash = (64 - 37) % 101 = 27
  
  Add 'D': text_hash = (256 * 27 + 68) % 101
        = (6912 + 68) % 101
        = 6980 % 101
        = 94
```

**Position i=3: Window "CDD"**
```
text_hash = 94
pattern_hash = 94
94 == 94 → Hash match! ✓

Verify character by character:
  text[3:6] = "CDD"
  pattern = "CDD"
  "CDD" == "CDD" ✓

Match found at position 3!
matches = [3]
```

**Continue sliding...**

```
Position 4: Window "DDA"
Position 5: Window "DAE"
Position 6: Window "AEF"
Position 7: Window "EFG"

None of these match (either hash mismatch or character mismatch)
```

**Final Result:** Match found at position `[3]`

**Visual Summary:**
```
Text:    A B C C D D A E F G
         [A B C]           hash=38, no match
           [B C C]         hash=75, no match
             [C C D]       hash=64, no match
               [C D D]     hash=94, MATCH! ✓
```

**Time Complexity:** 
- Average: O(n + m)
- Worst: O(n×m) (when many hash collisions)

**Key Advantage:** Very efficient when searching for multiple patterns simultaneously!

---

## **4. Boyer-Moore Algorithm**

### **Logic & Intuition**

**Key Ideas:**
1. **Bad Character Rule:** When mismatch occurs, skip alignments that can't possibly match
2. **Scan from right to left** in the pattern (opposite of other algorithms)
3. **Jump ahead** based on where the mismatched character appears in pattern

**Think of it like:** When reading, if you see a character that's not in the pattern at all, you can skip the entire pattern length ahead!

### **Code**

```python
def boyer_moore(text, pattern):
    n = len(text)
    m = len(pattern)
    
    if m == 0:
        return []
    
    # Build bad character table
    bad_char = build_bad_char_table(pattern)
    
    matches = []
    s = 0  # shift of the pattern with respect to text
    
    while s <= n - m:
        j = m - 1  # Start from rightmost character of pattern
        
        # Keep reducing j while characters match
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        
        # If pattern is found
        if j < 0:
            matches.append(s)
            # Shift pattern to align next character
            s += (m - bad_char.get(text[s + m], -1) - 1) if s + m < n else 1
        else:
            # Shift pattern based on bad character rule
            s += max(1, j - bad_char.get(text[s + j], -1))
    
    return matches

def build_bad_char_table(pattern):
    """
    Returns a dictionary with the rightmost position of each character
    """
    table = {}
    for i in range(len(pattern)):
        table[pattern[i]] = i
    return table
```

### **Dry Run: `text = "ABAAABCD"`, `pattern = "ABC"`**

**Step 1: Build Bad Character Table for "ABC"**

```
Pattern: A B C
Index:   0 1 2

Bad Character Table (rightmost occurrence):
{
  'A': 0,
  'B': 1,
  'C': 2
}
```

**Step 2: Search**

```
Text:    A B A A A B C D
Index:   0 1 2 3 4 5 6 7
```

**Alignment 1: s=0 (shift = 0)**
```
Text:    A B A A A B C D
Pattern: A B C
         ↑ ↑ ↑
         0 1 2

Start from right (j=2):
  text[0+2]='A' vs pattern[2]='C' ✗ Mismatch!
  
Bad character is 'A' at text position 2
'A' appears in pattern at index 0

Shift calculation:
  shift = max(1, j - bad_char['A'])
       = max(1, 2 - 0)
       = max(1, 2)
       = 2

New s = 0 + 2 = 2
```

**Visual:**
```
Text:    A B A A A B C D
Pattern: A B C (mismatch at C vs A)
             ↓ skip 2 positions
           A B C
```

**Alignment 2: s=2**
```
Text:    A B A A A B C D
Pattern:     A B C
             ↑ ↑ ↑
             0 1 2

Start from right (j=2):
  text[2+2]='A' vs pattern[2]='C' ✗ Mismatch!

'A' at position 4 appears in pattern at index 0

Shift = max(1, 2 - 0) = 2
New s = 2 + 2 = 4
```

**Alignment 3: s=4**
```
Text:    A B A A A B C D
Pattern:         A B C
                 ↑ ↑ ↑
                 0 1 2

Start from right (j=2):
  text[4+2]='C' vs pattern[2]='C' ✓ j=1
  text[4+1]='B' vs pattern[1]='B' ✓ j=0
  text[4+0]='A' vs pattern[0]='A' ✓ j=-1

j < 0 → Pattern found at position 4! ✓
matches = [4]

Calculate next shift:
  s + m = 4 + 3 = 7 < 8 (text length)
  Next char = text[7] = 'D'
  'D' not in pattern, so bad_char.get('D', -1) = -1
  
  Shift = m - (-1) - 1 = 3 - (-1) - 1 = 3
  New s = 4 + 3 = 7
```

**Alignment 4: s=7**
```
Text:    A B A A A B C D
Pattern:               A B C
                       ↑
                       
s=7, m=3, n=8
s + m = 10 > 8, so stop
```

**Final Result:** Pattern found at position `[4]`

**Complete Visualization:**
```
Text:    A B A A A B C D

Step 1:  A B C           (mismatch at C vs A, shift 2)
Step 2:      A B C       (mismatch at C vs A, shift 2)
Step 3:          A B C   (MATCH! ✓)
```

**Key Insight of Boyer-Moore:**
```
When we mismatch 'C' with 'A':
- Naive would shift by 1
- KMP would use LPS array
- Boyer-Moore looks at 'A' and says:
  "'A' is at position 0 in pattern, 
   and we're checking position 2,
   so we can skip 2-0 = 2 positions!"
```

**Another Example showing big jumps:**

```
Text:    GCTTCTGCTACCTTTTGCGCGCGCGCGCGC
Pattern: CCTTTTGC

Alignment 1:
Text:    GCTTCTGCTACCTTTTGCGCGCGCGCGCGC
Pattern: CCTTTTGC
                ↑
         Mismatch at 'C' vs 'G'
         'G' is not in pattern at all!
         → Can shift entire pattern length (8 positions)!

This is why Boyer-Moore is so fast for large patterns!
```

**Time Complexity:**
- Best: O(n/m) - can skip large chunks!
- Worst: O(n×m)
- Typically: Much better than naive in practice

---

## **Comparison of String Matching Algorithms**

| Algorithm | Time (Best) | Time (Avg) | Time (Worst) | Preprocessing | Best For |
|-----------|-------------|------------|--------------|---------------|----------|
| **Naive** | O(n) | O(n×m) | O(n×m) | None | Small patterns, teaching |
| **KMP** | O(n) | O(n+m) | O(n+m) | O(m) | Guaranteed linear time |
| **Rabin-Karp** | O(n+m) | O(n+m) | O(n×m) | O(m) | Multiple pattern search |
| **Boyer-Moore** | O(n/m) | O(n+m) | O(n×m) | O(m+σ)* | Large patterns, text editors |

*σ = alphabet size

### **When to Use Which?**

**Naive:**
- Pattern length < 5
- Learning/teaching
- One-time searches

**KMP:**
- Need guaranteed O(n) time
- DNA sequence matching
- When preprocessing pattern is cheap

**Rabin-Karp:**
- Searching multiple patterns simultaneously
- Plagiarism detection
- When hash collisions are rare

**Boyer-Moore:**
- Large patterns
- Text editors (Ctrl+F)
- Natural language text
- When pattern doesn't match often

**Python's `str.find()`:**
- Uses optimized version of Boyer-Moore-Horspool
- Best choice for production code!